In [ ]:
!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

!tar -xf v1.0-mini.tgz -C ./data/sets/nuscenes  # Uncompress the nuScenes mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

!pip install ultralytics #ultralytics que mantem o yolov8

In [2]:
epochs = 100
num_workers = 4
batch = 32

model = 'yolov8s.pt'

lrf_value = 0.0001
lr0_value = 0.0001
optimizer_value = 'SGD'

In [3]:
from nuscenes.nuscenes import NuScenes
import matplotlib.pyplot as plt
import cv2
import os
import matplotlib.patches as patches

from get_2D_boxes_from_sample_data import get_2D_boxes_from_sample_data
import json

In [4]:
data_path = './data/sets/nuscenes'
nusc = NuScenes(version='v1.0-mini', dataroot='./data/sets/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.408 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:
# Função que recebe o token da cena e retorna as bounding boxes 2D com anotações
from nuscenes.utils.geometry_utils import BoxVisibility

def get_boxes_from_specified_frame(nusc, frame_token):
    
    sample = nusc.get('sample', frame_token) # Obter o sample específico pelo token de frame fornecido
    
    sample_data = nusc.get('sample_data', sample['data']['CAM_FRONT']) # Trocar para outros sensores aqui quando necessário
        
    # Chamar a função para obter as bounding boxes 2D
    boxes_anns = get_2D_boxes_from_sample_data(nusc, sample_data['token'], box_vis_level=BoxVisibility.ANY, visibilities=['1', '2', '3', '4'])
    
    return boxes_anns


In [6]:
# Plotar as bounding boxes e anotações
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_boxes_on_image(image_path, boxes, annotations):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Criar uma figura e um eixo para plotar
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    
    # Plotar cada bounding box com sua anotação
    for box, label in zip(boxes, annotations):
        # Extrair coordenadas e dimensões da bounding box
        x_min, y_min, x_max, y_max = box
        width, height = x_max - x_min, y_max - y_min
        
        # Criar um retângulo para a bounding box
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='yellow', facecolor='none')
        ax.add_patch(rect)
        
        # Adicionar a etiqueta/label da bounding box
        ax.text(x_min, y_min, label, color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))
    
    plt.show()

In [7]:
# Inicializar o dicionário que armazenará os dados por cena
scenes_data = {}

for i in range(10): # Iterar sobre a qtd de cenas total
    scene = nusc.scene[i]
    scene_token = scene['token']  # Obter o token da cena atual
    frame_token = nusc.get('sample', scene['first_sample_token'])

    # Inicializar a lista que conterá os dados de todos os frames desta cena
    frames = []

    while frame_token['next'] != '':
        frame_token = nusc.get('sample', frame_token['next'])
        boxes, anns = get_boxes_from_specified_frame(nusc, frame_token['token'])
        image_path = data_path + '/' + nusc.get('sample_data', frame_token['data']['CAM_FRONT'])['filename']

        # Adicionar informações do frame na lista de frames
        frames.append({'boxes': boxes, 'anns': anns, 'path': image_path})

    # Atribuir a lista de frames à cena correspondente no dicionário
    scenes_data[scene_token] = frames

# Agora, scenes_data contém um dicionário onde cada chave é um token de cena,
# e o valor é uma lista de dicionários com dados dos frames.

#print(json.dumps(scenes_data, indent=4))


In [8]:
def convert_to_yolo_format(boxes, annotations, image_width, image_height, class_mapping):
    yolo_annotations = []

    for box, label in zip(boxes, annotations):
        x_min, y_min, x_max, y_max = box

        # Normalizar as coordenadas
        x_center = (x_min + x_max) / 2.0 / image_width
        y_center = (y_min + y_max) / 2.0 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        # Mapear a classe para um índice
        class_id = class_mapping[label]

        # Formato YOLO: <class_id> <x_center> <y_center> <width> <height>
        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_annotations


In [9]:
def getNuscClasses(): 
    category_names = {cat['token']: cat['name'] for cat in nusc.category}
    classes = {}
    i = 0
    for cat_token, cat_name in category_names.items():
        classes[cat_name] = i
        i += 1
    return classes


In [10]:
# Inserindo os dados YOLO formatados no dicionário de frames
class_mapping = getNuscClasses()

for scene_token in scenes_data:
    frames = scenes_data[scene_token]  # Lista de frames para a cena atual
    for frame in frames:
        # Obter a largura e altura da imagem
        img = cv2.imread(frame['path'])
        img_height, img_width, _ = img.shape

        # Converter as boxes e anotações para o formato YOLO
        yolo_formatted_annotations = convert_to_yolo_format(frame['boxes'], frame['anns'], img_width, img_height, class_mapping)

        # Adicionar as anotações YOLO formatadas ao dicionário do frame
        frame['yolo_anns'] = yolo_formatted_annotations

        # Imprimir as anotações YOLO para verificar
        # print("\nYOLO formatted annotations for frame: ", frame['path'], "in scene: ", scene_token)
        # for annotation in yolo_formatted_annotations:
        #     print(annotation)


#print(scenes_data)

Treinando o modelo de forma personalizada

In [11]:
from nuscenes.utils.splits import create_splits_scenes

splits = create_splits_scenes()

train_scenes = splits['mini_train']
val_scenes = splits['mini_val']

print("scene: ", scene)

print("Train scenes: ", train_scenes)
print("Validation scenes: ", val_scenes)
# token = nusc.field2token('scene', 'name', scene)[0]
# print("Token: ", token)
  

scene:  {'token': 'e233467e827140efa4b42d2b4c435855', 'log_token': '8fefc430cbfa4c2191978c0df302eb98', 'nbr_samples': 40, 'first_sample_token': 'a480496a5988410fbe3d8ed6c84da996', 'last_sample_token': 'abf3d91d3c28407e80e3334fe89c03cb', 'name': 'scene-1100', 'description': 'Night, peds in sidewalk, peds cross crosswalk, scooter, PMD, difficult lighting'}
Train scenes:  ['scene-0061', 'scene-0553', 'scene-0655', 'scene-0757', 'scene-0796', 'scene-1077', 'scene-1094', 'scene-1100']
Validation scenes:  ['scene-0103', 'scene-0916']


In [12]:
import os
import shutil
import cv2

def setup_yolo_training_environment(scenes_data, class_mapping, train_scenes, val_scenes):
    base_dir = './custom_train'
    train_image_dir = os.path.join(base_dir, 'images', 'train')
    train_label_dir = os.path.join(base_dir, 'labels', 'train')
    val_image_dir = os.path.join(base_dir, 'images', 'val')
    val_label_dir = os.path.join(base_dir, 'labels', 'val')

    # Criar diretórios
    os.makedirs(train_image_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_image_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)

    # Distribuir imagens e anotações entre treinamento e validação conforme as divisões de cenas
    for scene_token, frames in scenes_data.items():
        scene_name = nusc.get('scene', scene_token)['name']
        if scene_name in train_scenes:
            image_dir, label_dir = train_image_dir, train_label_dir
        elif scene_name in val_scenes:
            image_dir, label_dir = val_image_dir, val_label_dir
        else:
            print(f'Cena {scene_name} não está em train_scenes ou val_scenes')
            continue  # Pular cenas que não estão em train_scenes ou val_scenes

        for frame in frames:
            img_filename = os.path.basename(frame['path'])
            new_image_path = os.path.join(image_dir, img_filename)
            shutil.copy(frame['path'], new_image_path)

            label_filename = img_filename.replace('.jpg', '.txt')
            new_label_path = os.path.join(label_dir, label_filename)
            with open(new_label_path, 'w') as f:
                for annotation in frame['yolo_anns']:
                    f.write(annotation + '\n')

            print(f'Imagem copiada para: {new_image_path}')
            print(f'Anotações salvas em: {new_label_path}')

# Chamar a função de configuração usando as divisões específicas
setup_yolo_training_environment(scenes_data, class_mapping, train_scenes, val_scenes)

# Criar arquivo YAML
data_yaml_content = f"""path: {os.path.abspath('./custom_train')}
train: images/train
val: images/val

nc: {len(class_mapping)}
names: {list(class_mapping.keys())}
"""

yaml_path = './custom_train/data.yaml'
with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(data_yaml_content)

print(f'Arquivo YAML criado em: {yaml_path}')


Imagem copiada para: ./custom_train/images/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928112460.jpg
Anotações salvas em: ./custom_train/labels/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928112460.txt
Imagem copiada para: ./custom_train/images/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928662460.jpg
Anotações salvas em: ./custom_train/labels/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928662460.txt
Imagem copiada para: ./custom_train/images/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402929162460.jpg
Anotações salvas em: ./custom_train/labels/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402929162460.txt
Imagem copiada para: ./custom_train/images/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402929662460.jpg
Anotações salvas em: ./custom_train/labels/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402929662460.txt
Imagem copiada para: ./custom_train/images/train/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402930112

In [13]:
#esse bloco faz a substituição das classes de acordo com o ranking de classes do nuscenes, remova os blocos até o (from IPython import display) caso todas as labels devam ser mantidas
new_classes = {v: k for k, v in class_mapping.items()}

# remover classes : animal, human.pedestrian.personal_mobility, human.pedestrian.stroller, human.pedestrian.wheelchair, movable_object.debris, movable_object.pushable_pullable, static_object.bicycle_rack, vehicle.emergency.ambulance, vehicle.emergency.police
removedClasses = ['animal', 'human.pedestrian.personal_mobility', 'human.pedestrian.stroller', 'human.pedestrian.wheelchair', 'movable_object.debris', 'movable_object.pushable_pullable', 'static_object.bicycle_rack', 'vehicle.emergency.ambulance', 'vehicle.emergency.police']


pedestrians = ['human.pedestrian.adult', 'human.pedestrian.child', 'human.pedestrian.construction_worker', 'human.pedestrian.police_officer']
bus = ['vehicle.bus.bendy', 'vehicle.bus.rigid']

# removendo as classes de new_classes
for i in range(len(new_classes)):
    if new_classes[i] in removedClasses:
        new_classes.pop(i)
        
def replace_class_numbers_with_names(new_classes):
    dirs = ['./custom_train/labels/train', './custom_train/labels/val']
    for dir in dirs:
        for filename in os.listdir(dir):
            file_path = os.path.join(dir, filename)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            with open(file_path, 'w') as f:
                for line in lines:
                    if int(line.split(' ')[0]) in new_classes: #new classes ainda tem os indices de classes que serao aglutinadas
                        class_name = new_classes[int(line.split(' ')[0])]
                        if class_name in pedestrians:
                            class_name = 'pedestrian'
                        elif class_name in bus:
                            class_name = 'bus'
                        
                            
                        f.write(f"{class_name} {line[2:]}") #escreve o nome da classe e o resto da linha
                    else:
                        print(f"Classe {int(line[0])} não está em new_classes, removendo linha") #??
                
replace_class_numbers_with_names(new_classes)



Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 4 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 4 não está em new_classes, removendo linha
Classe 4 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha
Classe 2 não está em new_classes, removendo linha


In [14]:
# Remover classes que serão substituídas
classes_to_remove = ['human.pedestrian.adult', 'human.pedestrian.child', 'human.pedestrian.construction_worker', 'human.pedestrian.police_officer', 'vehicle.bus.bendy', 'vehicle.bus.rigid']
new_classes = {k: v for k, v in new_classes.items() if v not in classes_to_remove}

# Adicionar novas classes
new_classes[0] = 'pedestrian'
new_classes[1] = 'bus'

#ordene as classes pela chave
new_classes = dict(sorted(new_classes.items()))

# Reatribuir chaves sequenciais
sequential_new_classes = {i: v for i, v in enumerate(new_classes.values())}

# Mostrando o dicionário atualizado
print("Novo mapeamento de classes com chaves sequenciais:", sequential_new_classes)


Novo mapeamento de classes com chaves sequenciais: {0: 'pedestrian', 1: 'bus', 2: 'vehicle.car', 3: 'vehicle.motorcycle', 4: 'vehicle.bicycle', 5: 'vehicle.truck', 6: 'vehicle.construction', 7: 'vehicle.trailer', 8: 'movable_object.barrier', 9: 'movable_object.trafficcone'}


In [15]:
def replace_names_with_numbers(sequential_new_classes):
    dirs = ['./custom_train/labels/train', './custom_train/labels/val']
    for dir in dirs:
        for filename in os.listdir(dir):
            file_path = os.path.join(dir, filename)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            with open(file_path, 'w') as f:
                for line in lines:
                    class_name = line.split()[0]
                    class_number = list(sequential_new_classes.keys())[list(sequential_new_classes.values()).index(class_name)]
                    f.write(f"{class_number} {' '.join(line.split()[1:])}\n")

replace_names_with_numbers(sequential_new_classes)

In [16]:
def update_yaml_file(sequential_new_classes):
    with open(yaml_path, 'r') as f:
        data = f.readlines()

    data[4] = f"nc: {len(sequential_new_classes)}\n"
    data[5] = f"names: {list(sequential_new_classes.values())}\n"

    with open(yaml_path, 'w') as f:
        f.writelines(data)

update_yaml_file(sequential_new_classes)

In [16]:
# Rotina que divide e adiciona os dados do carla no 

In [17]:
from IPython import display
display.clear_output() # Limpar a saída para evitar poluição visual

import ultralytics
ultralytics.checks() # Verificar se o ambiente está configurado corretamente


Ultralytics YOLOv8.2.32 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Setup complete ✅ (12 CPUs, 15.6 GB RAM, 118.3/1006.9 GB disk)


In [18]:
!yolo task=detect mode=train model=$model plots=True exist_ok=True data=./custom_train/data.yaml epochs=$epochs optimizer=$optimizer_value lr0=$lr0_value lrf=$lrf_value batch=$batch workers=$num_workers

New https://pypi.org/project/ultralytics/8.2.64 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.32 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=./custom_train/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=True, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fal

In [19]:
# Validando o modelo por cli
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data=./custom_train/data.yaml exist_ok=True

Ultralytics YOLOv8.2.32 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Model summary (fused): 168 layers, 11129454 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /home/sprigganunix/Projetos/YOLO/custom_train/labels/val.cache... 
val: WARNING ⚠️ /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605512404.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0247]
val: WARNING ⚠️ /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151606012404.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3125      1.0039]
val: WARNING ⚠️ /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151607512404.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.8572      3.7161      2.6524]
val: WARNING ⚠️ /home/spri

In [20]:
# Testar o modelo treinado nas imagens de validacao (fica salvo em runs/predict)
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt conf=0.25 source=./custom_train/images/val exist_ok=True

Ultralytics YOLOv8.2.32 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
Model summary (fused): 168 layers, 11129454 parameters, 0 gradients, 28.5 GFLOPs

image 1/79 /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604012404.jpg: 384x640 6 pedestrians, 9 vehicle.cars, 67.4ms
image 2/79 /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604512404.jpg: 384x640 8 pedestrians, 4 vehicle.cars, 10.4ms
image 3/79 /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605012404.jpg: 384x640 6 pedestrians, 6 vehicle.cars, 1 movable_object.trafficcone, 9.8ms
image 4/79 /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605512404.jpg: 384x640 7 pedestrians, 9 vehicle.cars, 10.2ms
image 5/79 /home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-201

In [18]:
# Testar o modelo usando o python
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

# Carregar o modelo treinado
model = YOLO("runs/detect/train/weights/best.pt")

# Testar o modelo em uma imagem
results = model(["/home/sprigganunix/Projetos/YOLO/custom_train/images/val/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604012404.jpg"])

for result in results:
    # detection
    result.boxes.xyxy   # box with xyxy format, (N, 4)
    result.boxes.xywh   # box with xywh format, (N, 4)
    result.boxes.xyxyn  # box with xyxy format but normalized, (N, 4)
    result.boxes.xywhn  # box with xywh format but normalized, (N, 4)
    result.boxes.conf   # confidence score, (N, 1)
    result.boxes.cls    # cls, (N, 1)

    print("boxes da imagem: ", result.boxes.xyxy)
    print("confiança: ", result.boxes.conf)






0: 384x640 6 pedestrians, 9 vehicle.cars, 64.5ms
Speed: 5.4ms preprocess, 64.5ms inference, 134.3ms postprocess per image at shape (1, 3, 384, 640)
boxes da imagem:  tensor([[1.0079e+03, 4.7957e+02, 1.0746e+03, 5.3474e+02],
        [1.0950e+03, 4.7549e+02, 1.1485e+03, 5.1075e+02],
        [1.2924e+03, 4.6878e+02, 1.3349e+03, 5.6854e+02],
        [8.0092e+02, 4.8082e+02, 8.9403e+02, 5.4457e+02],
        [7.6656e+02, 4.7701e+02, 8.3926e+02, 5.3020e+02],
        [1.3490e+03, 4.6855e+02, 1.3963e+03, 5.6617e+02],
        [5.1822e+02, 4.7765e+02, 5.5849e+02, 5.6686e+02],
        [1.5164e+03, 4.3225e+02, 1.6000e+03, 6.3181e+02],
        [7.6902e+02, 4.7890e+02, 8.6262e+02, 5.3633e+02],
        [6.8531e+02, 4.8129e+02, 7.0431e+02, 5.2093e+02],
        [9.3888e+02, 4.6583e+02, 9.8692e+02, 5.0360e+02],
        [1.3622e+03, 4.7022e+02, 1.4032e+03, 5.6824e+02],
        [4.2161e-01, 5.2404e+02, 1.5263e+02, 5.9499e+02],
        [8.6667e+02, 4.6914e+02, 9.1791e+02, 4.9872e+02],
        [1.7583e+00, 

In [19]:
# Função para extrair as detecções do modelo YOLO (é possivel adicionar também o nivel de confiança no retorno)
def extract_detections(results, model):
    boxes_list = []
    labels_list = []
    
    # Assegura que model.names contém os nomes das classes mapeados pelo índice
    for result in results:
        # Extrair as boxes e a confiança
        boxes = result.boxes.xyxy.cpu().numpy()  # Convertendo para NumPy array e movendo para CPU
        confs = result.boxes.conf.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()  # Índices das classes

        # Iterar sobre cada box
        for box, conf, cls_idx in zip(boxes, confs, classes):
            x1, y1, x2, y2 = map(float, box) # Convertendo para float (atribuindo somente box originalmente me deu problema por ser float32, json nao aceitou)
            class_name = model.names[int(cls_idx)]  # Convertendo índice para nome da classe
            boxes_list.append((x1, y1, x2, y2))
            labels_list.append(class_name)

    return boxes_list, labels_list # boxes list aqui tem o formato xyxy, o que significa que x1 y1 é o canto superior esquerdo e x2 y2 é o canto inferior direito




In [20]:
json_content = {}
# no json_content deve ter o scene_token como chave, dentro disso uma lista de sample_data_token e dentro de cada um desses uma lista de anotações e boxes do yolo
for scene in val_scenes:
    scene_token = nusc.field2token('scene', 'name', scene)[0] # Buscando o token da cena
    cena = nusc.get('scene', scene_token) # Buscando a cena
    sample = nusc.get('sample', cena['first_sample_token']) # Buscando a primeira amostra da cena
    while sample['next'] != '':
        sample = nusc.get('sample', sample['next'])
        sample_data = nusc.get('sample_data', sample['data']['CAM_FRONT'])
        file = sample_data['filename']
        file = 'custom_train/images/val/' + file.split('/')[2]
        results = model([file]) # Detectando objetos na imagem com o yolo aqui
        boxes, labels = extract_detections(results, model)
        print(boxes)
        json_content[sample_data['token']] = {'boxes': boxes, 'labels': labels}





0: 384x640 6 pedestrians, 9 vehicle.cars, 11.7ms
Speed: 1.5ms preprocess, 11.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[(1007.8612060546875, 479.56695556640625, 1074.632080078125, 534.736083984375), (1094.9903564453125, 475.49066162109375, 1148.5283203125, 510.7513427734375), (1292.3961181640625, 468.781005859375, 1334.9388427734375, 568.5437622070312), (800.9182739257812, 480.81884765625, 894.031982421875, 544.5689697265625), (766.5601806640625, 477.0076904296875, 839.25537109375, 530.1986083984375), (1349.0322265625, 468.5491943359375, 1396.34423828125, 566.1681518554688), (518.2233276367188, 477.64959716796875, 558.4888305664062, 566.8567504882812), (1516.405029296875, 432.2511291503906, 1600.0, 631.81494140625), (769.0225219726562, 478.8990783691406, 862.6226806640625, 536.328125), (685.3142700195312, 481.2879638671875, 704.3060913085938, 520.9345703125), (938.875, 465.8316650390625, 986.920654296875, 503.59613037109375), (1362.1875, 470.2239990234375, 1

In [23]:
import json

# Salvar o conteúdo JSON
with open('detections.json', 'w') as f:
    json.dump(json_content, f, indent=4)

[ WARN:0@131.499] global loadsave.cpp:241 findDecoder imread_('custom_train/images/val/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604012404.jpg'): can't open/read file: check file path/integrity
[ WARN:0@131.500] global loadsave.cpp:241 findDecoder imread_('custom_train/images/val/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604512404.jpg'): can't open/read file: check file path/integrity
[ WARN:0@131.500] global loadsave.cpp:241 findDecoder imread_('custom_train/images/val/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605012404.jpg'): can't open/read file: check file path/integrity
[ WARN:0@131.500] global loadsave.cpp:241 findDecoder imread_('custom_train/images/val/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605512404.jpg'): can't open/read file: check file path/integrity
[ WARN:0@131.500] global loadsave.cpp:241 findDecoder imread_('custom_train/images/val/samples/CAM_FRONT/n008-2018-08-01-15-